In [1]:
#All Imports 
# !pip install datasets
# !pip install transformers
# !pip install SentencePiece

# !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

import torch
import transformers
from transformers import AutoTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric


from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# import nltk
# from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

# Transfer learning 
- train the pre-train model 

In [2]:
import pandas as pd
#mounting drive
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/NLP/Ass3/'

# article_df = pd.read_csv(data_dir + 'md_pmc_covid_articles.csv')
# article_df.head(5)
proc_article_df = pd.read_csv(data_dir + 'Processed_covid_articles.csv')
proc_article_df = proc_article_df.drop(['Unnamed: 0'], axis=1)
proc_article_df.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,paper_id,body_text,authors,title,abstract,Language
0,PMC8524215,cancer prevalent cause mortality world common ...,"['Muhammad Sufyan ', 'Farah Shahid ', 'Faiza...",implementation vaccinomics insilico approach c...,68591 One of the most common gynecologic ca...,en
1,PMC7698314,sarscov2 virus responsible covid19 disease fir...,"['Enrique Casalino ', 'Christophe Choquet ',...",analysis emergency department visit hospital a...,813557 ED-visits and through-ED admissions ...,en
2,PMC8050998,artificial intelligence ai methodology compute...,"['Martina Gurgitano ', 'Salvatore Alessio Ang...",interventional radiology exmachina impact arti...,757155 Artificial intelligence (AI) is a br...,en
3,PMC7581283,common feature patient admitted coronavirus di...,"['Ejvind Frausing Hansen ', 'Charlotte Sandau ...",automatic oxygen titration o2matic® patient ad...,942009 INTRODUCTION: Patients with coronavi...,en
4,PMC7392455,coronavirus disease covid19 caused severe acut...,"['Daniel O. Griffin ', 'Alexandra Jensen ', '...",pulmonary embolism increased level ddimer pati...,843714 We report 3 patients with coronaviru...,en


In [3]:
#Showing a sample 
sample = proc_article_df.iloc[0]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["body_text"])}):
""")
print(sample["body_text"][:500])
print(f'\nSummary (length: {len(sample["abstract"])}):')
print(sample["abstract"])


Article (excerpt of 500 characters, total length: 25277):

cancer prevalent cause mortality world common hindrance country achieving desired life expectancy . ovarian cancer one common type cancer woman . guleria . 2020 . ovarian cancer extremely lethal nastiest prognosis ailment . inability early diagnosis nearly nonspecific symptom common ovarian cancer identified iiiiv stage . five major histological form ovarian cancer identified epithelial edward . 2016 torre . 2018 siegel miller 2016 . endometrioid 13 mucinous 13 clear cell cancer 6 three prevalen

Summary (length: 88):
68591    One of the most common gynecologic cancers is ...
Name: abstract, dtype: object


In [4]:
import pandas as pd
# divide the dataframe into training data and test data
article_df_copy = proc_article_df.copy()
train_article = article_df_copy[['body_text','abstract']].iloc[:4000]  # 4000 rows for training
test_article = article_df_copy[['body_text','abstract']].iloc[4000:] # 500 rows for testing

train_article.head(5)

,body_text,abstract
0,cancer prevalent cause mortality world common ...,68591 One of the most common gynecologic ca...
1,sarscov2 virus responsible covid19 disease fir...,813557 ED-visits and through-ED admissions ...
2,artificial intelligence ai methodology compute...,757155 Artificial intelligence (AI) is a br...
3,common feature patient admitted coronavirus di...,942009 INTRODUCTION: Patients with coronavi...
4,coronavirus disease covid19 caused severe acut...,843714 We report 3 patients with coronaviru...


In [5]:
def tokenization(df,tokenizer):
    
    # tokenize in ids the body text and abstract
    tokenized_output = list()
    for i,j in zip(df['body_text'], df['abstract']):
        f_tokenized = tokenizer(i, padding="max_length", truncation=True)
        l_tokenized = tokenizer(j, padding="max_length", truncation=True)
        tokenized_text ={ 
        'feature_input_ids': f_tokenized['input_ids'].flatten(),
        'feature_attention_mask': f_tokenized['attention_mask'],
        'feature_text': i,
        'label_input_ids': l_tokenized['input_ids'].flatten(),
        'label_attention_mask': l_tokenized['attention_mask'],
        'label_text': j
        }
        
        tokenized_output.append(tokenized_text)
    return tokenized_output


## T5 model 

In [6]:
import transformers

model_name = 't5-small'
model = transformers.AutoModelWithLMHead.from_pretrained(model_name)

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

nlp_summarizer = transformers.pipeline('summarization', model = model, tokenizer = tokenizer)

nlp_summarizer(sample['body_text'])



/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding

[{'summary_text': 'ovarian cancer one common type of cancer identified iiiv stage . inability early diagnosis nearly nonspecific symptom common type cancer . hla allele significant since hold response .'}]

In [7]:
# import torch
# import transformers

# model_name = 't5-small'
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# t5_model = transformers.AutoModelWithLMHead.from_pretrained(model_name).to(device)
# t5_tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

# input_text = sample["body_text"]
# input_ids = t5_tokenizer.encode(input_text, truncation=True, padding="longest", return_tensors='pt', max_length=500).to(device)
# nlp_summarizer = transformers.pipeline('summarization', model=t5_model, tokenizer=t5_tokenizer, device=device.index if torch.cuda.is_available() else -1)
# output = nlp_summarizer(input_text)

## Pegasus Model

In [8]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

src_text = sample['body_text']

model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['ovarian cancer one common type cancer woman.']


In [15]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

src_text = sample['body_text']

model_name = "google/pegasus-large"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = PegasusTokenizer.from_pretrained(model_name)

model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt",max_length= 500).to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (256) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['endometrioid 13 mucinous 13 clear cell cancer 6 three prevalent type ovarian cancer coburn. benign ovarian tumor recommended threat indicator precursor epithelial ovarian cancer ovarian cancer benign ovarian tumor share equivalent etiological risk factor jordan sj green ac whiteman dc webb pm study ac 2007. clear cell endometrioid tumor wellthoughtout caused establishing endometrium ovary throughout retrograde menstruation kurman 2016. different reason predicting greater mortality rate ovarian cancer like nonsymptomatic stealthy progression malignant tumorous cell slow progression disease symptom lastly lacking specific method diagnosis cancer even advanced stage. 4.1 per 100000 compared 21290 case 14180 death related ovarian cancer usa 2015.']


In [10]:
sample['body_text']

'cancer prevalent cause mortality world common hindrance country achieving desired life expectancy . ovarian cancer one common type cancer woman . guleria . 2020 . ovarian cancer extremely lethal nastiest prognosis ailment . inability early diagnosis nearly nonspecific symptom common ovarian cancer identified iiiiv stage . five major histological form ovarian cancer identified epithelial edward . 2016 torre . 2018 siegel miller 2016 . endometrioid 13 mucinous 13 clear cell cancer 6 three prevalent type ovarian cancer coburn . 2017 . benign ovarian tumor recommended threat indicator precursor epithelial ovarian cancer ovarian cancer benign ovarian tumor share equivalent etiological risk factor jordan sj green ac whiteman dc webb pm study ac 2007 . also proposed fallopian tube may also contribute origination ovarian cancer distal end fallopian tube predecessor lesion ovary could progress severe ovarian tumor kurman 2016 . clear cell endometrioid tumor wellthoughtout caused establishing e

## Fine Tuning

In [12]:
sm_train_article = train_article.iloc[:10]
sm_train_article.head()

,body_text,abstract
0,cancer prevalent cause mortality world common ...,68591 One of the most common gynecologic ca...
1,sarscov2 virus responsible covid19 disease fir...,813557 ED-visits and through-ED admissions ...
2,artificial intelligence ai methodology compute...,757155 Artificial intelligence (AI) is a br...
3,common feature patient admitted coronavirus di...,942009 INTRODUCTION: Patients with coronavi...
4,coronavirus disease covid19 caused severe acut...,843714 We report 3 patients with coronaviru...


In [13]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

### convert to Huggingface dataset
sm_train_db = Dataset(pa.Table.from_pandas(sm_train_article))

In [14]:
sm_train_db

Dataset({
    features: ['body_text', 'abstract'],
    num_rows: 10
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

article_length = 512
summary_length = 64
batch_size     = 4
model_name = "google/pegasus-large"
# Load the BART's pre-trained Tokenizer
# TODO use pretrained model for medical data, covid would be best
# tokenizer = BartTokenizer.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
# Define the function to make the correct data structure
def tokenization(df,tokenizer):
    
    # tokenize in ids the body text and abstract
    tokenized_output = list()
    for i,j in zip(df['body_text'], df['abstract']):
        f_tokenized = tokenizer(i, padding="max_length", truncation=True)
        l_tokenized = tokenizer(j, padding="max_length", truncation=True)
        tokenized_text ={ 
        'feature_input_ids': f_tokenized['input_ids'].flatten(),
        'feature_attention_mask': f_tokenized['attention_mask'],
        'feature_text': i,
        'label_input_ids': l_tokenized['input_ids'].flatten(),
        'label_attention_mask': l_tokenized['attention_mask'],
        'label_text': j
        }
        
        tokenized_output.append(tokenized_text)
    return tokenized_output
    
def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=article_length)
  outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=summary_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # We have to make sure that the PAD token is ignored for calculating the loss
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

# Map the function to both train/validation sets.
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True,
    remove_columns=["article", "highlights"]
)

# Convert the Dataset to PyTorch tensor with the expected columns
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids",
                           "decoder_attention_mask", "labels"],
)

# Make the iterative object that does batching using the DataLoader
train_data      = DataLoader(train_data, batch_size=batch_size)

In [14]:
from torch.utils.data import DataLoader

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=16, device=device, 
                               column_text="body_text", 
                               column_summary="abstract"):
    article_batches = DataLoader(article_batches[column_text], batch_size=batch_size)
    target_batches = DataLoader(article_batches[column_summary], batch_size=batch_size)
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        
        # Finally, we decode the generated texts, 
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      
        
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
model_goog_peg_lrg = "google/pegasus-large"
pipe = pipeline('summarization', model = model_goog_peg_lrg )

pipe_out = pipe(train_article.iloc[0]['body_text'][:1024])

print(pipe_out)

In [16]:
train_article.iloc[0]['body_text']

'cancer prevalent cause mortality world common hindrance country achieving desired life expectancy . ovarian cancer one common type cancer woman . guleria . 2020 . ovarian cancer extremely lethal nastiest prognosis ailment . inability early diagnosis nearly nonspecific symptom common ovarian cancer identified iiiiv stage . five major histological form ovarian cancer identified epithelial edward . 2016 torre . 2018 siegel miller 2016 . endometrioid 13 mucinous 13 clear cell cancer 6 three prevalent type ovarian cancer coburn . 2017 . benign ovarian tumor recommended threat indicator precursor epithelial ovarian cancer ovarian cancer benign ovarian tumor share equivalent etiological risk factor jordan sj green ac whiteman dc webb pm study ac 2007 . also proposed fallopian tube may also contribute origination ovarian cancer distal end fallopian tube predecessor lesion ovary could progress severe ovarian tumor kurman 2016 . clear cell endometrioid tumor wellthoughtout caused establishing e

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(train_article, rouge_metric, model_pegasus, tokenizer, column_text = 'body_text', column_summary='abstract')

## Rough

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
temp = tokenizer(train_article['body_text'][0], padding="max_length", truncation=True)
temp.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenization(df,tokenizer):
    
    # tokenize in ids the body text and abstract
    tokenized_output = list()
    for i,j in zip(df['body_text'], df['abstract']):
        f_tokenized = tokenizer(i, padding="max_length", truncation=True)
        l_tokenized = tokenizer(j, padding="max_length", truncation=True)
        tokenized_text ={ 
        'feature_input_ids': f_tokenized['input_ids'].flatten(),
        'feature_attention_mask': f_tokenized['attention_mask'],
        'feature_text': i,
        'label_input_ids': l_tokenized['input_ids'].flatten(),
        'label_attention_mask': l_tokenized['attention_mask'],
        'label_text': j
        }
        
        tokenized_output.append(tokenized_text)
    return tokenized_output

# list of dictionaries, each dictionary is a tokenized article with 'input_ids', 'token_type_ids', 'attention_mask'

train_X_encode_ids = tokenization(train_article,tokenizer)
test_X_encode_ids = tokenization(test_article,tokenizer)

# train_y_encode_ids = tokenization(train_article, "abstract")
# test_y_encode_ids = tokenization(test_article, "abstract")



In [ ]:
train_X_encode_ids[0].keys()

dict_keys(['feature_input_ids', 'feature_attention_mask', 'feature_text', 'label_input_ids', 'label_attention_mask', 'label_text'])

In [ ]:
small_train_dataset = train_X_encode_ids[0:1000]
small_eval_dataset = test_X_encode_ids[0:1000]

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from transformers import BertForSequenceClassification
import torch
from torch.optim import Adam
from transformers import get_scheduler
import tqdm

l_rate = 1e-5
epochs = 2
train_steps = epochs * len(train_dataloader)
#Get transformaer specific model later
t_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = Adam(params = t_model.parameters(), lr=l_rate)

lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=train_steps
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
import torch
mps_device = torch.device("cpu")
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print("MPS device not found.")

tensor([1.], device='mps:0')


In [ ]:
def train(epochs, model, optimizer,train_dataloader):
    train_loss_ls = []
    train_accuracy_ls = []
    val_loss_ls = []
    val_accuracy_ls = []
    model = model.to(mps_device)
    #progress_bar = tqdm(range(train_steps))
    '''
     This is the training section
    '''
    
    for epoch in range(epochs):  # loop over the dataset multiple times
        
        training_loss = 0
        training_correct = 0
        model.train() # the model is in training mode, will consider evething like batch normalization and dropout
            
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            #progress_bar.update(1)

train(epochs, t_model, optimizer,train_dataloader)

            
        
        # for i, data in enumerate(train_dataloader, 0): # each data is one batch
        #     # get the inputs; data is a list of [inputs, labels]
        #     inputs, labels = data
        #     inputs, labels = inputs.to(mps_device), labels.to(mps_device)

        #     # zero the parameter gradients
        #     optimizer.zero_grad()

        #     # forward + backward + optimize
        #     outputs = model(inputs)
        #     predicted = outputs.argmax(1)# index of max probability out of 10 classes

        #     loss = criterion(outputs, labels)
        #     loss.backward()
        #     optimizer.step()

        #     # print statistics
        #     training_loss += loss.item()
        #     training_correct += (predicted == labels).sum().item()

#         '''
#         This is the validation section
# #         '''
#         val_loss = 0
#         val_correct = 0
# #         val_total = 0
#         model.eval()
#         with torch.no_grad(): # do not perform any calculation  
#             for data in test_dataloader:
#                 inputs, labels = data
#                 inputs, labels = inputs.to(device), labels.to(device)
#                 outputs = model(inputs)
#                 predicted = outputs.argmax(1)
#                 loss = criterion(outputs, labels)
#                 val_loss += loss.item()
# #                 val_total += labels.size(0)
#                 val_correct += (predicted == labels).sum().item()

# #         print('Val Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

#         training_loss = training_loss/(len(trainloader_resnet)) # this is consider as batch
#         train_acc = (training_correct / len(trainloader_resnet.dataset)) # this consider as all simple
#         val_loss = val_loss / len(validloader_resnet)
#         val_acc = (val_correct/len(validloader_resnet.dataset))
        
#         # save the loss and accuracy in the list
#         train_loss_ls.append(training_loss)
#         train_accuracy_ls.append(train_acc)
#         val_loss_ls.append(val_loss)
#         val_accuracy_ls.append(val_acc)
#         print("epoch",epoch)
#         print("train_accuracy: ",train_acc,"val_accuracy: ",val_acc,"train_loss: ",training_loss, "val_loss:", val_loss)

#     print('Finished Training')
#     return (train_accuracy_ls,train_loss_ls,val_accuracy_ls,val_loss_ls)


AttributeError: 'list' object has no attribute 'to'

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()